In [34]:
from PIL import Image
import pytesseract
import os

# Path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Path to the 'tessdata' directory
tessdata_path = r'C:\Program Files\Tesseract-OCR\tessdata'

# Specify the path to the image
image_path = 'C:\\Users\\Yaramis\\Documents\\GitHub\\DataScience\\Deep Learning\\ocr\\1673018557537.jpeg'

# Set the 'TESSDATA_PREFIX' environment variable for this script
os.environ['TESSDATA_PREFIX'] = tessdata_path

# Open the image using Pillow
img = Image.open(image_path)

# Perform OCR
text = pytesseract.image_to_string(img)

# Print the extracted text
print("Extracted Text:\n", text)

Extracted Text:
 GOLD STANDARD

:WHEY



In [37]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [38]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [43]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?crid=AD4666DY5KQC&k=whey%20protein&ref=glow_cls&refresh=1&sprefix=whey%2Caps%2C287"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [44]:
amazon_df

,title,price,rating,reviews,availability
0,"365 by Whole Foods Market, Vanilla Grass Fed W...",,4.2 out of 5 stars,346 ratings,Not Available
1,"Levels Grass Fed Whey Protein, No Artificials,...",,4.5 out of 5 stars,"14,248 ratings",Not Available
8,"Myprotein Impact Whey Protein, Chocolate Brown...",,4.4 out of 5 stars,"9,523 ratings",Not Available
10,"365 by Whole Foods Market, Chocolate Whey Prot...",,4.3 out of 5 stars,311 ratings,Not Available
32,"ALLMAX Nutrition AllWhey Classic, Chocolate, 5lb",,4.4 out of 5 stars,423 ratings,Not Available
37,ONNIT Grass Fed Whey Isolate Protein - Mexican...,,4.6 out of 5 stars,322 ratings,Not Available
38,"PERFECT Sports, Diesel 100% New Zealand Whey I...",,4.4 out of 5 stars,10 ratings,Not Available
39,"ALLMAX ISONATURAL Whey Protein Isolate, Chocol...",,4.6 out of 5 stars,610 ratings,Not Available
40,LEGION Whey+ Whey Isolate Protein Powder from ...,,4.4 out of 5 stars,83 ratings,Not Available
41,Vanilla Whey Protein Isolate Cold-Filtration -...,,4.4 out of 5 stars,334 ratings,Not Available
